In [2]:
# Requires sklearn 0.24; Install in AnacondaPrompt through: conda install scikit-learn=0.24
import numpy as np
import pandas as pd
import sklearn
from inspect import getmembers, isfunction
from rdkit import Chem
from rdkit.Chem import Descriptors

from tqdm.notebook import tqdm
# Add more if needed

# Data Preperation

In [3]:
# Import Data
data = pd.read_csv("train_descriptors.csv") # Replace with proper File
data_test = pd.read_csv("test_descriptors.csv")
train_crystals = pd.read_csv("train_crystals.csv")

In [4]:
# Test Data: From Row 13449
data_combined = data.append(data_test)
data_combined.head()

,identifiers,Unnamed: 0,name,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,...,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2,InchiKey,SMILES
0,COXSUE,11365,Optimised_Molecules_V3.3460,8.773011,8.632564,0,0,14.599164,2.244739,4.388162,...,185.062283,8.046186,219,11,54.0,58.0,4.833333,2.777778,UYSCRWBEPUAZRA-UHFFFAOYSA-N,CC(=O)NN=C(C)C1=NCCS1
1,TUWFAU,58476,Optimised_Molecules_V3.19496,10.813451,10.346393,0,0,18.488707,2.357869,4.715737,...,199.110947,7.111105,364,22,70.0,80.0,6.805556,3.583333,RYPSYCXCJSHDJX-UHFFFAOYSA-N,CC(C)N(C)c1ccc(C#N)cc1C#N
2,DUTHEG,31879,Optimised_Molecules_V3.5074,10.922841,9.584172,0,0,16.685332,2.436274,4.872549,...,191.080710,8.307857,271,22,74.0,87.0,5.666667,3.000000,PVRUUABKIQNQFL-UHFFFAOYSA-N,Cc1cc(=O)n2c(N)nc(C)nc2n1
3,AXUDED08,2885,Optimised_Molecules_V3.988,7.140017,6.873356,0,0,12.857279,2.214320,4.428639,...,136.052429,7.558468,121,11,44.0,48.0,3.722222,2.444444,QPJVMBTYPHYUOC-UHFFFAOYSA-N,COC(=O)c1ccccc1
4,ARAGUV,2660,Optimised_Molecules_V3.795,7.399237,7.308033,0,0,11.638727,2.249778,4.311276,...,139.074562,7.319714,125,8,46.0,49.0,4.333333,2.250000,LUFRABHJXNJTNZ-UHFFFAOYSA-N,CC(=O)Nc1cc(C)[nH]n1


In [5]:
train_crystals.head()

,identifiers,n_heavy_atoms,smiles,a,b,c,alpha,beta,gamma,z_value,...,cell_volume,calculated_density,packing_coefficient,void_volume,is_centrosymmetric,is_sohncke,vdw_inter,vdw_intra,hbond_inter,hbond_intra
0,COXSUE,12,CC(=O)NN=C(C)C1=NCCS1,4.7362,17.0760,11.4150,90.0,94.432,90.0,4.0,...,920.431558,1.336802,0.667769,0.0,True,False,8,6,1,0
1,TUWFAU,15,CC(C)N(C)c1ccc(cc1C#N)C#N,8.9832,12.6212,9.6863,90.0,92.414,90.0,4.0,...,1097.246124,1.206163,0.689272,0.0,True,False,4,16,0,0
2,DUTHEG,14,CC1=CC(=O)N2C(=NC(=NC2=N1)C)N,11.1369,18.9130,4.0311,90.0,90.000,90.0,4.0,...,849.079420,1.495633,0.729064,0.0,False,False,7,4,1,1
3,AXUDED08,10,COC(=O)c1ccccc1,13.1840,5.9231,16.9510,90.0,90.000,90.0,8.0,...,1323.706139,1.366336,0.728081,0.0,True,False,4,4,0,0
4,ARAGUV,10,CC(=O)NC1=NNC(=C1)C,18.6000,19.9320,7.7992,90.0,90.000,90.0,16.0,...,2891.437972,1.278655,0.660441,0.0,False,False,5,2,2,0


In [6]:
# Clean all columns with more than 60% empty;
threshold = 0.6
data_cleaned_comb = data_combined[data_combined.columns[(data_combined==0).mean() < threshold]]
data_cleaned_comb.head()

,identifiers,Unnamed: 0,name,ABC,ABCGG,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,...,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2,InchiKey,SMILES
0,COXSUE,11365,Optimised_Molecules_V3.3460,8.773011,8.632564,14.599164,2.244739,4.388162,14.599164,1.216597,...,185.062283,8.046186,219,11,54.0,58.0,4.833333,2.777778,UYSCRWBEPUAZRA-UHFFFAOYSA-N,CC(=O)NN=C(C)C1=NCCS1
1,TUWFAU,58476,Optimised_Molecules_V3.19496,10.813451,10.346393,18.488707,2.357869,4.715737,18.488707,1.232580,...,199.110947,7.111105,364,22,70.0,80.0,6.805556,3.583333,RYPSYCXCJSHDJX-UHFFFAOYSA-N,CC(C)N(C)c1ccc(C#N)cc1C#N
2,DUTHEG,31879,Optimised_Molecules_V3.5074,10.922841,9.584172,16.685332,2.436274,4.872549,16.685332,1.191809,...,191.080710,8.307857,271,22,74.0,87.0,5.666667,3.000000,PVRUUABKIQNQFL-UHFFFAOYSA-N,Cc1cc(=O)n2c(N)nc(C)nc2n1
3,AXUDED08,2885,Optimised_Molecules_V3.988,7.140017,6.873356,12.857279,2.214320,4.428639,12.857279,1.285728,...,136.052429,7.558468,121,11,44.0,48.0,3.722222,2.444444,QPJVMBTYPHYUOC-UHFFFAOYSA-N,COC(=O)c1ccccc1
4,ARAGUV,2660,Optimised_Molecules_V3.795,7.399237,7.308033,11.638727,2.249778,4.311276,11.638727,1.163873,...,139.074562,7.319714,125,8,46.0,49.0,4.333333,2.250000,LUFRABHJXNJTNZ-UHFFFAOYSA-N,CC(=O)Nc1cc(C)[nH]n1


In [7]:
# Deal with Outliers: 
# Cap all data points below 5 and above 95% to value at 5 and 95% respectively
def cap_outliers(data, column, upper = 0.95, lower=0.05):
    upper_lim = data[column].quantile(upper)
    lower_lim = data[column].quantile(lower)
        
    data.loc[(data[column] > upper_lim),column] = upper_lim
    data.loc[(data[column] < lower_lim),column] = lower_lim
    

In [8]:
# Print all Columns to inspect which are categorical
for i, name in enumerate(data_cleaned_comb.columns):
    print("{}: {}".format(i, name))

0: identifiers
1: Unnamed: 0
2: name
3: ABC
4: ABCGG
5: SpAbs_A
6: SpMax_A
7: SpDiam_A
8: SpAD_A
9: SpMAD_A
10: LogEE_A
11: VE1_A
12: VE2_A
13: VE3_A
14: VR1_A
15: VR2_A
16: VR3_A
17: nAromAtom
18: nAromBond
19: nAtom
20: nHeavyAtom
21: nHetero
22: nH
23: nC
24: nN
25: nO
26: ATS0dv
27: ATS1dv
28: ATS2dv
29: ATS3dv
30: ATS4dv
31: ATS5dv
32: ATS6dv
33: ATS7dv
34: ATS0d
35: ATS1d
36: ATS2d
37: ATS3d
38: ATS4d
39: ATS5d
40: ATS6d
41: ATS7d
42: ATS8d
43: ATS0s
44: ATS1s
45: ATS2s
46: ATS3s
47: ATS4s
48: ATS5s
49: ATS6s
50: ATS7s
51: ATS8s
52: ATS0Z
53: ATS1Z
54: ATS2Z
55: ATS3Z
56: ATS4Z
57: ATS5Z
58: ATS6Z
59: ATS7Z
60: ATS8Z
61: ATS0m
62: ATS1m
63: ATS2m
64: ATS3m
65: ATS4m
66: ATS5m
67: ATS6m
68: ATS7m
69: ATS8m
70: ATS0v
71: ATS1v
72: ATS2v
73: ATS3v
74: ATS4v
75: ATS5v
76: ATS6v
77: ATS7v
78: ATS8v
79: ATS0se
80: ATS1se
81: ATS2se
82: ATS3se
83: ATS4se
84: ATS5se
85: ATS6se
86: ATS7se
87: ATS8se
88: ATS0pe
89: ATS1pe
90: ATS2pe
91: ATS3pe
92: ATS4pe
93: ATS5pe
94: ATS6pe
95: ATS7pe
96

In [9]:
# Ignore all categorical Columns and cap Outliers
# Add/Remove as needed
categorical_columns = [
        "identifiers", "Unnamed: 0" , "name", "InchiKey", "SMILES"
]  

for name in data_cleaned_comb.columns:
    if name in categorical_columns:
        continue
    cap_outliers(data_cleaned_comb, name)

C:\Users\Marvin\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [10]:
# Deal with missing values
# 1. Show which columns have missing data
pd.set_option('display.max_rows', None)
print(data_cleaned_comb.isnull().sum())
pd.set_option('display.max_rows', 10)

identifiers           0
Unnamed: 0            0
name                  0
ABC                   0
ABCGG                 0
SpAbs_A               0
SpMax_A               0
SpDiam_A              0
SpAD_A                0
SpMAD_A               0
LogEE_A               0
VE1_A                 0
VE2_A                 0
VE3_A                 0
VR1_A                 0
VR2_A                 0
VR3_A                 1
nAromAtom             0
nAromBond             0
nAtom                 0
nHeavyAtom            0
nHetero               0
nH                    0
nC                    0
nN                    0
nO                    0
ATS0dv                0
ATS1dv                0
ATS2dv                0
ATS3dv                0
ATS4dv                0
ATS5dv                0
ATS6dv                0
ATS7dv                0
ATS0d                 0
ATS1d                 0
ATS2d                 0
ATS3d                 0
ATS4d                 0
ATS5d                 0
ATS6d                 0
ATS7d           

In [11]:
# 2. Replace all with Median
data_cleaned_comb = data_cleaned_comb.fillna(data.median())

In [12]:
# Drop identifiers
data_cleaned_comb_drop = data_cleaned_comb.drop(categorical_columns, axis=1)
data_cleaned_comb_drop.head()

,ABC,ABCGG,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,VE1_A,VE2_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,8.773011,8.632564,14.599164,2.244739,4.388162,14.599164,1.216597,3.370933,3.078270,0.256523,...,8.764834,54.650723,185.062283,8.046186,219.0,11.0,54.0,58.0,4.833333,2.777778
1,10.813451,10.346393,18.488707,2.357869,4.715737,18.488707,1.232580,3.592641,3.459673,0.230645,...,9.421492,46.551655,199.110947,7.111105,364.0,22.0,70.0,80.0,6.805556,3.500000
2,10.922841,9.584172,16.685332,2.436274,4.872549,16.685332,1.191809,3.577079,3.471281,0.247949,...,9.693445,46.214439,191.080710,8.307857,271.0,22.0,74.0,87.0,5.666667,3.000000
3,7.140017,6.873356,12.857279,2.214320,4.428639,12.857279,1.285728,3.188884,2.963010,0.296301,...,8.703009,38.792641,136.052429,7.558468,121.0,11.0,44.0,48.0,3.722222,2.444444
4,7.399237,7.308033,11.638727,2.249778,4.311276,11.638727,1.163873,3.200300,2.910476,0.291048,...,8.616314,52.270726,139.074562,7.319714,125.0,8.0,46.0,49.0,4.333333,2.250000


In [13]:
# Convert from pandas dataframe to numpy array
data_np = data_cleaned_comb_drop.to_numpy()

In [14]:
#Scale X data to make it digestible to the models
from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit(data_np) # make Scaler
data_np = scaler.transform(data_np) # Apply scaler to data
data_np

C:\Users\Marvin\anaconda3\lib\site-packages\sklearn\utils\extmath.py:847: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
C:\Users\Marvin\anaconda3\lib\site-packages\sklearn\utils\extmath.py:689: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = op(x, *args, **kwargs)


array([[-0.03255635,  0.20143948,  0.0121117 , ..., -0.29550846,
         0.01844574,  0.10423566],
       [ 0.92987744,  1.17697404,  1.09562754, ...,  0.82578357,
         1.5548775 ,  1.37014114],
       [ 0.98147436,  0.74310746,  0.59325861, ...,  1.1825583 ,
         0.66764226,  0.49374504],
       ...,
       [-2.11784209, -1.98106206, -1.86008416, ..., -2.07938213,
        -0.7389502 , -1.40511317],
       [ 0.91643006,  0.5604968 ,  0.90645061, ...,  0.77481575,
        -0.28451264,  0.68849973],
       [ 0.02686056,  0.34402116,  0.01265841, ...,  0.41804101,
         1.44667808, -0.09051902]])

In [15]:
# Handle nan or inf
data_np = np.nan_to_num(data_np, posinf=1.5, neginf=-1.5)

In [16]:
X_train = data_np[:13449]
X_test = data_np[13449:]

In [ ]:
np.savetxt("X_train_cleaned.txt", X_train)

In [ ]:
np.savetxt("X_test_cleaned.txt", X_test)